In [2]:
!python -c 'import torch;print(torch.__version__)'

1.10.1


In [3]:
import numpy as np
from mmcv.transforms import Resize
transform = Resize(scale=(224, 224))
data_dict = {'img': np.random.rand(256, 256, 3)}
data_dict = transform(data_dict)
print(data_dict['img'].shape)

(224, 224, 3)


In [4]:
# pipeline = [
#     dict(type='LoadImageFromFile'),
#     dict(type='Resize', size=256, keep_ratio=True),
#     dict(type='CenterCrop', crop_size=224),
#     dict(type='Normalize', mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375]),
#     dict(type='ClsFormatBundle')
# ]

img_norm_cfg = dict(
     mean=[124.508, 116.050, 106.438],
     std=[58.577, 57.310, 57.437],
     to_rgb=False)

train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='RandomResizedCrop', scale=100, backend='pillow'),
    dict(type='Normalize', **img_norm_cfg),
    dict(type='ImageToTensor', keys=['img']),
    dict(type='ToTensor', keys=['gt_label']),
    dict(type='Collect', keys=['img', 'gt_label']),
    dict(type='ClsFormatBundle')
]
test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='Resize', scale=(100, -1), backend='pillow'),
    dict(type='CenterCrop', crop_size=100),
    dict(type='Normalize', **img_norm_cfg),
    dict(type='ImageToTensor', keys=['img']),
    dict(type='Collect', keys=['img']),
    dict(type='ClsFormatBundle')
]

train_dataloader = dict(
    batch_size=32,
    num_workers=2,
    dataset=dict(
        type='CustomDataset',
        data_prefix='data/YoriDataset_vgg/train',
        classes='data/classes.txt',
        ann_file='data/train_ann.txt',
        with_label=True,
        pipeline=train_pipeline        
    ),
)

val_dataloader = dict(
    batch_size=32,
    num_workers=2,
    dataset=dict(
        type='CustomDataset',
        data_prefix='data/YoriDataset_vgg/validation',
        classes='data/classes.txt',
        ann_file='data/val_ann.txt',
        with_label=True,
        pipeline=test_pipeline
    ),
)

test_dataloader = dict(
    batch_size=32,
    num_workers=2,
    dataset=dict(
        type='CustomDataset',
        data_prefix='data/YoriDataset_vgg/test',
        classes='data/classes.txt',
        ann_file='data/test_ann.txt',
        with_label=True,
        pipeline=test_pipeline
    ),    
)


In [5]:
model = dict(
    type='ImageClassifier',
    backbone=dict(
        type='ResNet_CIFAR',
        depth=18,
        num_stages=4,
        out_indices=(3, ),
        style='pytorch'),
    neck=dict(type='GlobalAveragePooling'),
    head=dict(
        type='LinearClsHead',
        num_classes=10,
        in_channels=512,
        loss=dict(type='L1Loss', loss_weight=1.0),
        topk=(1, 5),
    ))

In [7]:
optim_wrapper = dict(
    type='OptimWrapper',
    optimizer = dict(
        type='Adam',
        lr=0.001,
        betas=(0.9, 0.999),
        eps=1e-08,
        weight_decay=0,
        amsgrad=False),
)

param_scheduler = [
      # linear warm-up by iters
      dict(type='LinearLR',
          start_factor=0.001,
          by_epoch=False,  # by iters
          end=50),  # only warm up for first 50 iters
      # main learing rate schedule
      dict(type='MultiStepLR',
          by_epoch=True,
          milestones=[8, 11],
          gamma=0.1)
  ]